In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('persona.csv')

# first few rows

In [3]:
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


# about dimesions(rows and columns)

In [4]:
df.shape

(5000, 5)

# information about dataset


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


# How many unique cities are there? 

In [9]:
print(df['SOURCE'].nunique())

2


# What are their frequencies?


In [10]:
print(df['SOURCE'].value_counts())

android    2974
ios        2026
Name: SOURCE, dtype: int64


# How many unique Price are there?

In [11]:
print(df['PRICE'].nunique())

6


# How many sales were realized from which Price?

In [12]:
print(df['PRICE'].value_counts())

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64


# How much was earned in total from sales by Country?

In [18]:
df.groupby('COUNTRY')['PRICE'].count()

COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

# How much was earned in total from sales by country?

In [17]:
df.groupby('COUNTRY')['PRICE'].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

# What are the sales numbers by SOURCE types?

In [19]:
df['SOURCE'].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

# What are the PRICE averages by country?

In [23]:
df.groupby(by=['COUNTRY']).aggregate({'PRICE' : 'mean'})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


# What are the PRICE averages by SOURCEs?

In [24]:
df.groupby(by=['SOURCE']).aggregate({'PRICE' : 'mean'})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


# What are the PRICE averages in the COUNTRY-SOURCE breakdown?

In [25]:
df.groupby(by=['SOURCE', 'COUNTRY']).aggregate({'PRICE' : 'mean'})

PRICE
SOURCE  COUNTRY           
android bra      34.387029
        can      33.330709
        deu      33.869888
        fra      34.312500
        tur      36.229437
        usa      33.760357
ios     bra      34.222222
        can      33.951456
        deu      34.268817
        fra      32.776224
        tur      33.272727
        usa      34.371703

# What are the average earnings in breakdown of COUNTRY, SOURCE, SEX, AGE?

In [26]:
df.groupby(by=['SOURCE', 'COUNTRY', 'SEX', 'AGE']).aggregate({'PRICE' : 'mean'})

PRICE
SOURCE  COUNTRY SEX    AGE           
android bra     female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
ios     usa     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

# Sort the output by PRICE.

In [28]:
agg_df = df.groupby(by=['SOURCE', 'COUNTRY', 'SEX', 'AGE']).aggregate({'PRICE' : 'mean'}).sort_values("PRICE", ascending = False)
agg_df.head()

PRICE
SOURCE  COUNTRY SEX    AGE       
android fra     female 24    59.0
        usa     male   36    59.0
        bra     male   46    59.0
ios     usa     male   32    54.0
android deu     female 36    49.0

# Convert the names in the index to variable names.

In [29]:
agg_df = agg_df.reset_index()
agg_df.head()

,SOURCE,COUNTRY,SEX,AGE,PRICE
0,android,fra,female,24,59.0
1,android,usa,male,36,59.0
2,android,bra,male,46,59.0
3,ios,usa,male,32,54.0
4,android,deu,female,36,49.0


# Convert age variable to categorical variable and add it to agg_df.

In [31]:
agg_df['AGE'].describe()

count    348.000000
mean      28.258621
std       11.379075
min       15.000000
25%       19.000000
50%       25.000000
75%       34.000000
max       66.000000
Name: AGE, dtype: float64

In [32]:
bins = [0, 18, 23, 30, 40, agg_df['AGE'].max()]
mylabels = ['0_18', '19_23', '24_30', '31_40', '41_' + str(agg_df['AGE'].max())]

In [33]:
agg_df['age_cat'] = pd.cut(agg_df['AGE'], bins, labels = mylabels)
agg_df.head()

,SOURCE,COUNTRY,SEX,AGE,PRICE,age_cat
0,android,fra,female,24,59.0,24_30
1,android,usa,male,36,59.0,31_40
2,android,bra,male,46,59.0,41_66
3,ios,usa,male,32,54.0,31_40
4,android,deu,female,36,49.0,31_40


# Identify new level-based customers (personas).

In [35]:
agg_df['customers_level_based'] = agg_df[['COUNTRY', 'SOURCE', 'SEX', 'age_cat']].agg(lambda x: '_'.join(x).upper(), axis=1)
agg_df.head()

,SOURCE,COUNTRY,SEX,AGE,PRICE,age_cat,customers_level_based
0,android,fra,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
1,android,usa,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,android,bra,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
3,ios,usa,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,android,deu,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


# Segment new customers (personas).

In [36]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head(30)

,SOURCE,COUNTRY,SEX,AGE,PRICE,age_cat,customers_level_based,SEGMENT
0,android,fra,female,24,59.000000,24_30,FRA_ANDROID_FEMALE_24_30,A
1,android,usa,male,36,59.000000,31_40,USA_ANDROID_MALE_31_40,A
2,android,bra,male,46,59.000000,41_66,BRA_ANDROID_MALE_41_66,A
3,ios,usa,male,32,54.000000,31_40,USA_IOS_MALE_31_40,A
4,android,deu,female,36,49.000000,31_40,DEU_ANDROID_FEMALE_31_40,A
5,android,deu,female,51,49.000000,41_66,DEU_ANDROID_FEMALE_41_66,A
6,android,usa,female,47,49.000000,41_66,USA_ANDROID_FEMALE_41_66,A
7,ios,deu,male,20,49.000000,19_23,DEU_IOS_MALE_19_23,A
8,ios,fra,male,20,49.000000,19_23,FRA_IOS_MALE_19_23,A
9,ios,usa,male,59,46.500000,41_66,USA_IOS_MALE_41_66,A


In [37]:
agg_df.groupby("SEGMENT").agg({"PRICE": "mean"})

,PRICE
SEGMENT,
D,27.302596
C,32.933339
B,35.436170
A,41.434736


# Classify new customers and estimate how much revenue they can generate.

In [39]:
new_user = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,SOURCE,COUNTRY,SEX,AGE,PRICE,age_cat,customers_level_based,SEGMENT
19,android,tur,female,32,43.000000,31_40,TUR_ANDROID_FEMALE_31_40,A
35,android,tur,female,31,40.666667,31_40,TUR_ANDROID_FEMALE_31_40,A


In [40]:
new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,SOURCE,COUNTRY,SEX,AGE,PRICE,age_cat,customers_level_based,SEGMENT
206,ios,fra,female,40,33.000000,31_40,FRA_IOS_FEMALE_31_40,C
222,ios,fra,female,31,32.636364,31_40,FRA_IOS_FEMALE_31_40,C
